Logistic Regression CLF



In [ ]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
import xgboost as xgb
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import FeatureHasher
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import f1_score
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import datetime

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from string import punctuation
from gensim import models
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
import re

In [ ]:
!kaggle competitions download -c nlp-getting-started

sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!ls

catboost_info	       kaggle.json	      test.csv	       XGBoost.ipynb
CatBoost.ipynb	       resultados.csv	      train.csv
colsample_bylevel.png  sample_submission.csv  Untitled0.ipynb


In [ ]:
train=pd.read_csv("train.csv",encoding='latin-1')
datos =pd.read_csv("test.csv",encoding='latin-1')
datos.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
#Separo el target de los tweets
target = train.iloc[:,-1]

In [ ]:
#Aplico operaciones sobre train que luego replicare en el set de datos, para obtener los indices que yo considero relevantes
#y deshacerme de los que no lo son
#Empiezo por las arroba
train['cantidad@']=train['text'].str.count('@')
datos['cantidad@']=datos['text'].str.count('@')
train.head()

,id,keyword,location,text,target,cantidad@
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0


In [ ]:
#Aplico una funcion de hash sobre la location
#Primero fijo los NaN como la cadena "ninguna"
#Luego proceso eliminando mayusculas haciendo lemmatize
import spacy
def reemplazarTexto(x):
  x=x.casefold()
  return x

def procesarLocation(df):
  df['location']=df['location'].fillna(value='none')
  return df['location'].apply(lambda x:reemplazarTexto(x))
train['location']=procesarLocation(train)
datos['location']=procesarLocation(datos)

nlp = spacy.load("en_core_web_sm")
def proc(texts):
  loc=''
  if (texts=='none'):
    return 'none'
  tokens=nltk.word_tokenize(texts)
  for doc in nlp.pipe(tokens, disable=["tagger"]):
      for ent in doc.ents:
        if ([(ent.text,ent.label_)]!=[]) & (ent.label_=='GPE'):
          loc+=ent.text
  if (loc==[]):
    return 'no valida'
  return loc
train['locationLimpia']=train['location'].apply(lambda x:proc(x))
datos['locationLimpia']=datos['location'].apply(lambda x:proc(x))

In [ ]:
train['ubicacionValida']= ((train['locationLimpia']!='no valida') & (train['locationLimpia']!='none'))*1
datos['ubicacionValida']= ((datos['locationLimpia']!='no valida') & (datos['locationLimpia']!='none'))*1

In [ ]:
train['ubicacionValida'].describe()

count    7613.000000
mean        0.667280
std         0.471218
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: ubicacionValida, dtype: float64

In [ ]:
#VERIFICAR QUE LOS LOCATION SEAN DE UBICACIONES VERDADERAS
"""paises=['afghanistan', 'albania', 'algeria', 'andorra', 'angola', 'antigua & barbuda', 'argentina', 'armenia', 'australia', 'austria','azerbaijan', 'bahamas', 'bahrain','bangladesh','barbados', 'belarus', 'belgium', 'belize', 'benin', 'bhutan', 'bolivia', 
       'bosnia & herzegovina', 'botswana', 'brazil', 'bulgaria', 'burkina faso', 'burundi', 'cabo verde', 'cambodia', 'cameroon', 
       'canada', 'chad', 'chile', 'china', 'colombia', 'comoros', 'congo', 'costa rica', 'croatia', 'cuba', 'cyprus', 'czech republic', 
       'denmark', 'djibouti', 'dominica', 'dominican republic', 'ecuador', 'egypt', 'el salvador', 'equatorial guinea', 'eritrea', 'estonia', 
       'eswatini', 'fiji', 'finland', 'france', 'gabon', 'gambia', 'georgia', 'germany', 'ghana', 'greece', 'grenada', 'guatemala', 
       'guinea', 'guinea-bissau', 'guyana', 'haiti', 'honduras', 'hungary', 'iceland', 'india', 'indonesia', 'iran', 
       'iraq', 'ireland', 'israel', 'italy', 'jamaica', 'japan', 'jordan', 'kazakhstan', 'kenya', 'kosovo', 'kuwait', 
       'kyrgyzstan', 'laos', 'latvia', 'lebanon', 'lesotho', 'liberia', 'libya', 'liechtenstein', 'lithuania', 'luxembourg', 
       'madagascar', 'malawi', 'malaysia kuala lumpur', 'maldives male', 'mali', 'malta', 'marshall', 'islands', 'mauritania', 'mauritius', 
       'mexico', 'micronesia', 'moldova', 'monaco', 'mongolia', 'montenegro', 'morocco', 'mozambique', 'myanmar', 'namibia', 'nepal', 
       'netherlands', 'new zealand', 'nicaragua', 'niger', 'nigeria', 'norway', 'pakistan', 
       'palestine', 'panama', 'papua new guinea', 'paraguay', 'peru', 'philippines', 'poland', 'portugal', 'qatar', 'romania', 
       'russia moscow', 'rwanda kigali', 'saudi arabia', 'serbia', 'seychelles', 'sierra leone', 'singapore', 'slovakia', 'slovenia','south africa', 'south korea', 
       'south sudan juba', 'spain', 'sri lanka', 'sudan', 'suriname', 'sweden ', 'switzerland bern', 'syria', 'tajikistan','thailand', 'turkey', 'turkmenistan', 'uganda', 'ukraine', 'united arab emirates', 'uae',
       'united kingdom', 'uk', 'united states of america', 'usa', 'uruguay', 'uzbekistan', 
       'vatican city', 'holy see', 'venezuela',  'vietnam', 'yemen', 'zambia','zimbabwe']
capitales=['kabul','tirana','algiers','andorra la vella','luanda','buenos aires','yerevan','canberra','vienna','baku','manama','dhaka','minsk','bridgetown','brussels',
           'belmopan','porto-novo','thimphu','sucre','sarajevo','gaborone','brasilia','sofia','bujumbura','praia','phnom penh','yaounde','ottawa','santiago','beijing','bogotá',' moroni','san jose',
           ' zagreb','havana','nicosia','prague','copenhagen','roseau','santo domingo','quito','cairo','san salvador','malabo','asmara','tallinn','mbabane','suva','helsinki','paris','libreville','tbilisi','berlin',
           'accra','athens','guatemala city','mexico city','mexico df','conakry','bissau','georgetown','port-au-prince','tegucigalpa','budapest','reykjavik','new delhi','jakarta','tehran','baghdad','dublin','jerusalem',
           'rome','kingston','tokyo','amman','nairobi','pristina','kuwait city','bishkek','vientiane','riga','beirut','maseru','monrovia','tripoli','vaduz','vilnius','luxembourg','antananarivo','lilongwe',
           'monaco','podgorica','bamako','valletta','majuro','nouakchott','windhoek','rabat','maputo','kathmandu','port louis','amsterdam','wellington','managua','niamey','abuja','colombo','belgrade',
           'panama city','east jerusalem','port moresby','asunción','lima','madrid','khartoum','paramaribo','stockholm','freetown','singapore','caracas','damascus','dushanbe','harare','bangkok',
           'ankara','bucharest','riyadh','manila','warsaw','oslo','islamabad','lisbon','victoria','bratislava','seoul','kiev','london','abu dhabi','washington','washington dc','washington d.c.', 'd.c.','montevideo','tashkent', 'vatican city',
           'hanoi']
estadosUS= ["alabama","alaska","arizona","arkansas","california","colorado",\
  "connecticut","delaware","florida","georgia","hawaii","idaho","illinois",\
  "indiana","iowa","kansas","kentucky","louisiana","maine","maryland",\
  "massachusetts","michigan","minnesota","mississippi","missouri","montana",\
  "nebraska","nevada","new hampshire","new jersey","new mexico","new york",\
  "north carolina","north dakota","ohio","oklahoma","oregon","pennsylvania",\
  "rhode island","south carolina","south dakota","tennessee","texas","utah",\
  "vermont","virginia","washington","west virginia","wisconsin","wyoming"]
train['locationPais']=train['location'].str.contains('|'.join(paises))*1
datos['locationPais']=datos['location'].str.contains('|'.join(paises))*1
train['locationCiudad']=(train['location'].str.contains('|'.join(capitales)) | train['location'].str.contains('|'.join(estadosUS)))*1
datos['locationCiudad']=(datos['location'].str.contains('|'.join(capitales)) | datos['location'].str.contains('|'.join(estadosUS)))*1"""
train['location'].unique()


array(['none', 'birmingham', 'est. september 2012 - bristol', ...,
       'vancouver, canada', 'london ', 'lincoln'], dtype=object)

In [ ]:
#Analizo longitud de los tweets
train['longitudTweet']=train.text.str.len()
datos['longitudTweet']=datos.text.str.len()

In [ ]:
#Analizo la cantidad de Hashtags en un tweet
train['cantidad#']=train['text'].str.count('#')
datos['cantidad#']=datos['text'].str.count('#')

In [ ]:
#Cuento si el texto contiene un http y cuantos tiene
def analizarHttp(x):
  tokens=nltk.word_tokenize(x)
  resultado=0
  for token in tokens:
    resultado += ((token=='http') | (token=='https'))*1      
  return resultado

train['cantidadLinks']=train['text'].apply(lambda x:analizarHttp(x))
datos['cantidadLinks']=datos['text'].apply(lambda x:analizarHttp(x))

In [ ]:
def tokenizarTexto(x):
  x=x.casefold()
  x=re.sub(r'http\S*', '',x)
  x=re.sub(r'[^a-z\s]', '',x)
  lemma=WordNetLemmatizer()
  x=lemma.lemmatize(x)
  #tokens=nltk.word_tokenize(x)
  tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
  tokens = tknzr.tokenize(x)
  l=stopwords.words('english')+list(punctuation)
  tokens = [word for word in tokens if word not in l]
  return tokens
def docToVec (train,datos):
  l=[]
  datosTweets=datos.apply(lambda x:[tokenizarTexto(x['text']),x['id']],axis=1).tolist()
  trainTweets=train.apply(lambda x:[tokenizarTexto(x['text']),x['id']],axis=1).tolist()
  for vectores in datosTweets:
    l.append(TaggedDocument(words=vectores[0], tags=str(vectores[1])))
  for vectores in trainTweets:
    l.append(TaggedDocument(words=vectores[0], tags=str(vectores[1])))
  max_epochs = 350
  vec_size = 200
  model = Doc2Vec(vector_size=vec_size, epochs=max_epochs)  
  model.build_vocab(l)
  model.train(l,total_examples=model.corpus_count,epochs=max_epochs)
  ldatos=[]
  ltrain=[]
  for vectores in datosTweets:
    ldatos.append(model.infer_vector(vectores[0]))
  dfdatos= pd.DataFrame(ldatos)
  for vectores in trainTweets:
    ltrain.append(model.infer_vector(vectores[0]))
  dftrain= pd.DataFrame(ltrain)
  return (dfdatos,dftrain)

(dfdatos,dftrain) = docToVec(train,datos)
datos=pd.concat([datos,dfdatos],axis=1)
train=pd.concat([train,dftrain],axis=1)

In [ ]:
train.head(1)

,id,keyword,location,text,target,cantidad@,locationLimpia,ubicacionValida,longitudTweet,cantidad#,cantidadLinks,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,...,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199
0,1,NaN,none,Our Deeds are the Reason of this #earthquake M...,1,0,none,0,69,1,0,-0.628512,0.327033,-0.304269,-0.731991,0.287174,0.156645,0.353764,-0.39049,-0.131508,0.215879,-0.186127,-1.03127,0.118385,0.070875,0.132564,0.496785,-0.612761,-0.594813,0.287288,0.233444,-0.489822,0.455912,-0.056925,-0.194851,0.129908,0.484518,0.021995,0.364827,0.518986,...,-0.953195,-0.264285,-0.28194,0.223585,0.203874,-0.587167,-0.398523,0.1809,0.053667,-0.203619,0.536191,-0.057141,-0.308006,0.056288,-0.391408,0.649311,-0.74968,0.237442,-0.018645,0.385283,0.186466,-0.199661,0.030966,0.189886,0.511104,0.251037,0.418892,-0.54567,0.020075,-0.237423,0.554051,0.179571,0.439526,-0.011126,-0.870365,0.421312,-0.456047,-0.320762,-0.124312,-0.072145


In [ ]:
#Proceso cada hashtag
def procesarHashtag(x):
  tokens=nltk.word_tokenize(x)
  listaHashtags=[]
  for i in range(len(tokens)):
    if ((tokens[i]=='#') & (i!=len(tokens)-1)):
      listaHashtags.append(tokens[i])
  return listaHashtags

train['hashtagsVector']=train['text'].apply(lambda x:procesarHashtag(x))
datos['hashtagsVector']=datos['text'].apply(lambda x:procesarHashtag(x))
#Hago TF IDF


In [ ]:
#Analizo si los hashtags contienen alguna keyword
def analizarHashtag(x,keys):
  tokens=nltk.word_tokenize(x)
  lemma=WordNetLemmatizer()
  resultado=False
  for i in range(len(tokens)):
    if ((tokens[i]=='#') & (i!=len(tokens)-1)):
      resultado = lemma.lemmatize(tokens[i+1]) in keys
  return resultado*1
lTrainK=train['keyword'].unique().tolist()
lDatosK=train['keyword'].unique().tolist()
train['hashtagKey'] = train['text'].apply(lambda x:analizarHashtag(x,lTrainK))
datos['hashtagKey'] = datos['text'].apply(lambda x:analizarHashtag(x,lDatosK))

In [ ]:
#Analizo si se menciona algun tipo de entretenimiento en el tweet
#Puede ser juegos, musica, peliculas...
listaEntretenimiento=['music','radio','tv','show','dj','game','gaming','videogame','history','movies','youtube','book','ebook','sports','esports','http']

def analizarContenido(x):
  tokens=nltk.word_tokenize(x)
  resultado=0
  for palabra in tokens:
    if palabra in listaEntretenimiento:
      resultado = 1
  return resultado

train['entretenimiento'] = train['text'].apply(lambda x:analizarContenido(x))
datos['entretenimiento'] = datos['text'].apply(lambda x:analizarContenido(x))

In [ ]:
#Hago TF IDF para keywords

In [ ]:
#Analizo la relacion entre cantidad de # y la longitud del tweet
train['relacion#longitud']=train['cantidad#']/train['longitudTweet']*100
datos['relacion#longitud']=datos['cantidad#']/datos['longitudTweet']*100

In [ ]:
#Analizo la relacion entre cantidad de @ y la longitud del tweet
train['relacion@longitud']=train['cantidad@']/train['longitudTweet']*100
datos['relacion@longitud']=datos['cantidad@']/datos['longitudTweet']*100

In [ ]:
train['relacionTodoLongitud']=(train['cantidad@']+train['cantidad#']+train['cantidadLinks'])/train['longitudTweet']*100
datos['relacionTodoLongitud']=(datos['cantidad@']+datos['cantidad#']+datos['cantidadLinks'])/datos['longitudTweet']*100

In [ ]:
#Hago hashing trick para keywords
train['keyword'].fillna(value='none',inplace=True)
datos['keyword'].fillna(value='none',inplace=True)
def crearFeatureHasher(serie,ndf):
  fh = FeatureHasher(n_features=6, input_type='string')
  sp = fh.fit_transform(serie)
  df = pd.DataFrame(sp.toarray(), columns=['kh1', 'kh2', 'kh3', 'kh4', 'kh5', 'kh6'])
  ndf=pd.concat([ndf, df], axis=1)
  return ndf
train=crearFeatureHasher(train['keyword'],train)
datos=crearFeatureHasher(datos['keyword'],datos)

In [ ]:
train['location'].fillna(value='none',inplace=True)
train['location'].fillna(value='none',inplace=True)
def crearFeatureHasher(serie,ndf):
  fh = FeatureHasher(n_features=6, input_type='string')
  sp = fh.fit_transform(serie)
  df = pd.DataFrame(sp.toarray(), columns=['fh1', 'fh2', 'fh3', 'fh4', 'fh5', 'fh6'])
  ndf=pd.concat([ndf, df], axis=1)
  return ndf
train=crearFeatureHasher(train['location'],train)
datos=crearFeatureHasher(datos['location'],datos)
train.head()

,id,keyword,location,text,target,cantidad@,locationLimpia,ubicacionValida,longitudTweet,cantidad#,cantidadLinks,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,...,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,hashtagsVector,hashtagKey,entretenimiento,relacion#longitud,relacion@longitud,relacionTodoLongitud,kh1,kh2,kh3,kh4,kh5,kh6,fh1,fh2,fh3,fh4,fh5,fh6
0,1,none,none,Our Deeds are the Reason of this #earthquake M...,1,0,none,0,69,1,0,-0.628512,0.327033,-0.304269,-0.731991,0.287174,0.156645,0.353764,-0.390490,-0.131508,0.215879,-0.186127,-1.031270,0.118385,0.070875,0.132564,0.496785,-0.612761,-0.594813,0.287288,0.233444,-0.489822,0.455912,-0.056925,-0.194851,0.129908,0.484518,0.021995,0.364827,0.518986,...,-0.018645,0.385283,0.186466,-0.199661,0.030966,0.189886,0.511104,0.251037,0.418892,-0.545670,0.020075,-0.237423,0.554051,0.179571,0.439526,-0.011126,-0.870365,0.421312,-0.456047,-0.320762,-0.124312,-0.072145,[#],1,0,1.449275,0.0,1.449275,-2.0,2.0,0.0,0.0,0.0,0.0,-2.0,2.0,0.0,0.0,0.0,0.0
1,4,none,none,Forest fire near La Ronge Sask. Canada,1,0,none,0,38,0,0,-0.129401,0.466667,0.199845,-0.351677,0.383968,-0.373357,0.353000,-0.294482,0.129295,-0.424057,0.067760,-0.332570,0.064428,-0.253274,-0.417641,0.373122,-0.946637,-0.898118,-0.394234,-0.013864,-0.489987,0.410968,-0.166592,-0.147610,0.189896,0.740537,-0.197006,0.632659,0.191287,...,0.141572,0.191068,0.112340,0.168036,-0.084536,-0.399606,0.084875,0.083916,-0.017628,0.297720,0.013026,-0.255449,-0.204412,0.029893,0.363902,-0.814501,-0.325460,0.494968,0.282198,-0.724554,0.261204,-0.454392,[],0,0,0.000000,0.0,0.000000,-2.0,2.0,0.0,0.0,0.0,0.0,-2.0,2.0,0.0,0.0,0.0,0.0
2,5,none,none,All residents asked to 'shelter in place' are ...,1,0,none,0,133,0,0,-0.896472,0.036172,0.173572,-0.266737,0.331849,0.166484,0.600858,-0.512161,-0.605263,-0.328444,0.522028,-0.232392,0.405028,0.103294,0.114704,0.915332,-0.965241,-0.723623,0.317709,0.602889,-0.793480,0.276510,-0.455217,0.023144,0.262825,0.397481,0.427847,0.036423,0.834915,...,0.200126,-0.191071,0.040041,0.102310,0.203359,-0.048928,1.009853,0.413955,0.250676,-0.093168,-0.294752,-0.375655,-0.119350,0.040490,0.728681,-0.315963,-0.505727,-0.037074,-0.323861,-0.190092,0.392548,-0.224910,[],0,0,0.000000,0.0,0.000000,-2.0,2.0,0.0,0.0,0.0,0.0,-2.0,2.0,0.0,0.0,0.0,0.0
3,6,none,none,"13,000 people receive #wildfires evacuation or...",1,0,none,0,65,1,0,-0.107583,0.148694,0.062378,-0.421343,-0.011294,0.239749,0.375849,-0.383890,-0.307825,-0.125802,0.040969,-0.326797,-0.218666,-0.433647,-0.053203,0.243344,-0.819520,-0.889129,-0.067736,0.214459,-0.396157,0.113379,-0.015251,-0.167585,0.067060,-0.035390,-0.167043,0.019922,0.537810,...,-0.048257,0.298279,0.190215,0.253391,0.051187,-0.254962,0.059886,0.202076,-0.049089,-0.560714,0.025176,-0.488451,-0.022918,0.417538,0.206081,-0.202561,-0.211296,0.169655,-0.072453,-0.140722,0.151615,-0.057637,[#],1,0,1.538462,0.0,1.538462,-2.0,2.0,0.0,0.0,0.0,0.0,-2.0,2.0,0.0,0.0,0.0,0.0
4,7,none,none,Just got sent this photo from Ruby #Alaska as ...,1,0,none,0,88,2,0,-0.992190,-0.274439,0.326517,-0.292516,-0.038879,0.734120,-0.059260,-0.519631,-0.247928,-0.318836,0.608495,-0.852269,0.043739,-0.116989,-0.435518,0.284727,-0.853398,-1.022297,-0.474693,0.277045,-0.497995,0.568948,-0.453472,-0.118863,0.220772,0.419338,-0.116952,0.499869,0.305810,...,0.635083,0.145986,0.018226,0.029805,-0.148586,0.413789,0.392134,0.201352,0.225536,0.053016,-0.038981,-0.349619,0.115398,0.358548,0.371098,-0.267741,-0.208853,0.389986,-0.286518,0.182312,-0.335305,0.239747,"[#, #]",1,0,2.272727,0.0,2.272727,-2.0,2.0,0.0,0.0,0.0,0.0,-2.0,2.0,0.0,0.0,0.0,0.0


In [ ]:
#Remuevo cosas que no hacen al analisis
entrenar=train.drop(columns=['target','id','keyword','location','text','relacion#longitud','relacion@longitud','longitudTweet','cantidad@','cantidad#','hashtagsVector','locationLimpia'])
entrenar =entrenar.apply(lambda x: (x*1))
test=datos.drop(columns=['id','keyword','location','text','relacion#longitud','relacion@longitud','longitudTweet','cantidad@','cantidad#','hashtagsVector','locationLimpia'])
test =test.apply(lambda x: (x*1),axis=1)

In [ ]:
entrenar.describe()

,ubicacionValida,cantidadLinks,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,hashtagKey,entretenimiento,relacionTodoLongitud,kh1,kh2,kh3,kh4,kh5,kh6,fh1,fh2,fh3,fh4,fh5,fh6
count,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,...,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000,7613.000000
mean,0.667280,0.619467,-0.765761,0.294390,0.209784,-0.557558,0.338669,-0.117124,0.392048,-0.424293,-0.280991,-0.254183,0.169124,-0.641654,0.078308,-0.017316,-0.245074,0.588462,-1.162278,-0.804112,0.021108,0.296186,-0.700187,0.456716,-0.190864,-0.093724,0.017636,0.405742,-0.076349,0.465189,0.375134,-0.124591,-0.184955,-0.028590,-0.107000,-0.358641,0.655595,0.051834,0.037788,-0.711824,...,0.449348,0.361104,-0.020656,0.203600,0.278817,0.260558,0.068513,0.329622,-0.009170,0.566764,0.156750,0.410511,-0.272572,-0.065083,-0.371376,0.092324,0.226487,0.365404,-0.204684,-0.492409,0.134832,-0.243928,-0.373352,0.230061,-0.121804,0.006699,0.485485,1.402383,-0.647051,2.349796,0.947721,-1.397478,0.660975,-0.749507,-1.418101,2.443058,1.883751,-1.702351,0.594772,-0.767634
std,0.471218,0.663851,0.338854,0.284110,0.338412,0.295086,0.271513,0.258507,0.293194,0.283810,0.281166,0.274364,0.287051,0.306262,0.272353,0.264715,0.291418,0.301509,0.373128,0.333363,0.265111,0.255805,0.331646,0.309532,0.298829,0.267289,0.254859,0.293184,0.217340,0.294924,0.288084,0.293193,0.241163,0.255392,0.272636,0.282899,0.322064,0.285996,0.243933,0.357393,...,0.314169,0.294995,0.270227,0.261578,0.272432,0.259117,0.265551,0.296613,0.260799,0.288644,0.289752,0.310904,0.276185,0.256290,0.298651,0.255145,0.296753,0.310833,0.260547,0.317358,0.257639,0.257200,0.270427,0.271363,0.253107,0.081579,0.499822,1.322129,0.789000,1.492207,1.145440,1.334357,0.746684,1.060401,1.052996,1.931994,2.491502,2.032433,0.918433,1.253788
min,0.000000,0.000000,-2.172898,-0.853555,-1.106606,-2.149628,-0.656113,-1.718288,-0.906099,-1.577474,-1.683887,-1.555000,-1.271589,-2.256207,-1.231876,-1.341631,-1.933924,-0.534809,-2.832319,-2.282245,-1.244675,-0.964740,-2.447252,-0.609833,-2.270563,-1.213602,-1.639336,-0.743534,-1.107547,-1.403704,-0.838206,-1.751679,-1.367615,-1.249569,-2.316448,-1.941155,-0.968261,-1.713461,-1.162890,-2.589968,...,-1.097294,-1.439557,-1.411032,-1.514756,-1.196133,-0.966348,-0.988732,-0.866721,-1.307482,-0.826876,-1.358291,-1.193244,-2.222184,-1.273624,-2.013701,-1.215152,-1.194910,-2.012861,-1.891749,-1.868757,-1.253925,-2.092933,-1.749987,-1.525343,-1.212954,0.000000,0.000000,0.000000,-3.000000,-1.000000,-2.000000,-5.000000,0.000000,-4.000000,-7.000000,-11.000000,-21.000000,-14.000000,-1.000000,-10.000000
25%,0.000000,0.000000,-0.971098,0.096263,-0.024058,-0.748208,0.151099,-0.269535,0.198609,-0.604126,-0.460266,-0.428810,-0.011261,-0.837010,-0.086084,-0.170807,-0.424822,0.380017,-1.405054,-1.017591,-0.133307,0.124326,-0.909714,0.246205,-0.365151,-0.259279,-0.138695,0.202038,-0.208537,0.270389,0.188336,-0.306429,-0.335788,-0.185773,-0.265872,-0.535877,0.436347,-0.103895,-0.111435,-0.941912,...

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


clf = LogisticRegression(random_state=0,max_iter=500)
#Construimos los datos de entrenamiento y de test
X = entrenar
y = train["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


# estandarizamos las features
scaler = StandardScaler()

# Don't cheat - fit only on training data
scaler.fit(X_train)  # doctest: +SKIP
X_train = scaler.transform(X_train)  # doctest: +SKIP
# apply same transformation to test data
X_test = scaler.transform(X_test)  # doctest: +SKI
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 73.41%


In [ ]:
scaler = StandardScaler()

# Don't cheat - fit only on training data
scaler.fit(entrenar)  # doctest: +SKIP
x_train= scaler.transform(entrenar)  # doctest: +SKIP
# apply same transformation to test data
clf.fit(x_train, target)
test=scaler.fit_transform(test)
preds = clf.predict(test)
predictions = pd.DataFrame()
predictions['id']=datos['id']
predictions['target']=preds
predictions.head()
predictions.to_csv('resultados.csv',index=False)

In [ ]:
predictions.head()

,id,target
0,0,0
1,2,0
2,3,1
3,9,0
4,11,1


In [ ]:
#!kaggle competitions submit -c nlp-getting-started -f resultados.csv -m "Nuevo intento"

100% 22.2k/22.2k [00:01<00:00, 17.3kB/s]
Successfully submitted to Real or Not? NLP with Disaster Tweets